***GENERATED CODE FOR clusteringkmeans PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
import json
from sklearn.cluster import KMeans
import numpy as np
from sklearn import metrics
import pandas as pd


def kmeans(df, TrainingPercent, k):
    rows = df.count()
    percentrow = TrainingPercent / 100 * rows
    percentrows = int(percentrow)
    df = df.limit(percentrows)
    pd_df = df.toPandas()
    model = KMeans(n_clusters=k).fit(pd_df)
    labels = model.predict(pd_df)
    silhouette = metrics.silhouette_score(pd_df, labels, metric='euclidean')
    pd_df['prediction'] = pd.DataFrame(labels)
    labels_unique = np.unique(labels)
    totalClusters = len(labels_unique)
    display("totalClusters     : %s" % totalClusters)
    display(pd_df.head())
    return pd_df


class Clustering:

    def run(spark_DF, spark, config):
        stageAttributes = json.loads(config)
        stageAttributes['model']
        trainingPercent = stageAttributes['TrainingPercent']
        clusters = stageAttributes['totalClusters']

        bkmeans_model = kmeans(
            df=spark_DF, TrainingPercent=trainingPercent, k=clusters)


***READING DATAFRAME***

In [ ]:
#%run clusteringkmeansHooks.ipynb
try:
	#sourcePreExecutionHook()

	clusterlocations = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/1614001948670_uber_routing_LanLon.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line', 'FilePath': '/BigLynx/uber_routing_LanLon.csv', 'filename': '1614001948670_uber_routing_LanLon.csv', 'viewFileName': 'uber_routing_LanLon.csv'}")

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run clusteringkmeansHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(clusterlocations,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerLat", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "32.42", "stddev": "1.32", "min": "29.3199333", "max": "40.5854106", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CustomerLat"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustomerLon", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-97.15", "stddev": "2.32", "min": "-106.576434", "max": "-73.9650892", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CustomerLon"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "StoreLat", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "32.38", "stddev": "1.34", "min": "29.3052293", "max": "40.7105133", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "StoreLat"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "StoreLon", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-97.18", "stddev": "2.32", "min": "-106.5854384", "max": "-73.8917059", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "StoreLon"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run clusteringkmeansHooks.ipynb
try:
	#mlPreExecutionHook()

	kmean = Clustering.run(autofe,spark,json.dumps( {"autoClustering": 0, "originalfile": "/FileStore/platform/sampleData/6034e4da15b1ce3b4f942b41/part-00000-3de121d2-1cf3-4928-b84d-e1c06ff4e436-c000.csv", "model": "K-Means", "TrainingPercent": 100, "totalClusters": 80}))

	#mlPostExecutionHook(kmean)

except Exception as ex: 
	logging.error(ex)
